In [ ]:
%cd ..
import os
import time

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA_FULL = "NUSCENES_PROCESSED_DATA_FULL"
if NUSCENES_PROCESSED_DATA_FULL in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA_FULL]
else:
    DATA_DIR = "/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Trainval"
NUSCENES_RAW_DATA_FULL = "NUSCENES_RAW_DATA_FULL"
if NUSCENES_RAW_DATA_FULL in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA_FULL]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval"

In [ ]:
import cv2
def construct_video(frames, size=(1600, 900), base=False, vid_name=None):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig15_"+vid_name if vid_name else f"./outputs/{'fig15_base' if base else 'fig15_optimized'}.avi"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [ ]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

cam = camera
car1 = objects[0]
opposite_car = objects[1]
car2 = objects[2]

world = world.filter(
    (F.like(car1.type, 'car') | F.like(car1.type, 'truck')) &
    (F.like(car2.type, 'car') | F.like(car2.type, 'truck')) &
    (F.like(opposite_car.type, 'car') | F.like(opposite_car.type, 'truck')) &
    (opposite_car.id != car2.id) &
    (car1.id != car2.id) &
    (car1.id != opposite_car.id) &

    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
    (F.view_angle(car1.traj@cam.time, cam.ego) < 70 / 2) &
    (F.distance(cam.ego, car1.traj@cam.time) < 50) &
#     F.angle_between(F.facing_relative(car1.traj@cam.time, cam.ego), -15, 15) &
#     F.angle_between(F.facing_relative(car1.traj@cam.time, F.road_direction(car1.traj@cam.time, cam.ego)), -15, 15) &
    F.ahead(car1.traj@cam.time, cam.ego) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
    (F.convert_camera(opposite_car.traj@cam.time, cam.ego) > [-10, 0]) &
    (F.convert_camera(opposite_car.traj@cam.time, cam.ego) < [-1, 50]) &
    F.angle_between(F.facing_relative(opposite_car.traj@cam.time, cam.ego), 140, 180) &
    (F.distance(opposite_car@cam.time, car2@cam.time) < 40) &
#     F.angle_between(F.facing_relative(car2.traj@cam.time, F.road_direction(car2.traj@cam.time, cam.ego)), -15, 15) &
    F.ahead(car2.traj@cam.time, opposite_car.traj@cam.time)
)


In [ ]:
base_query_start = time.time()
preprocess(world, DATA_DIR, ['scene-0207-CAM_FRONT'])
print(f'base query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base = world.get_id_time_camId_filename(3)

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[5] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
preprocess(world, DATA_DIR, ['scene-0207-CAM_FRONT'], base=False)

In [ ]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(3)
print(f'optimized query time: {time.time()-optimized_query_start}')


In [ ]:
optimized_results = [e[5] for e in id_time_camId_filename]
construct_video(optimized_results)

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")

In [ ]:
investigation = [[5, 806.0, 414.0, 95.0, 121.0], [17, 186.0, 312.0, 395.0, 339.0], [37, 645.0, 475.0, 76.0, 63.0]]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open('/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval/samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg')

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(im)

# Create a Rectangle patch
for obj in investigation:
    obj_id = obj[0]
    x,y,w,h = obj[1:5]
    plt.text(x, y+15, 'obj'+str(obj_id))
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()